In [ ]:
# -------------------------------------------------
# 1️⃣ 구글 드라이브 마운트
# -------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 39 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:

# === 프로젝트 루트 (네 폴더 위치에 맞춰 바꿔줘) ===
# 예) MyDrive/colab_projects/pill_detect_runs 처럼 본인 폴더 경로로 지정
PROJECT_DIR = "/content/drive/MyDrive/pill_det/mnt_data"

import os
from pathlib import Path

# 확인용 출력
print("PROJECT_DIR:", PROJECT_DIR)
print("존재?", os.path.isdir(PROJECT_DIR))
print("안의 파일 10개 예시:", list(Path(PROJECT_DIR).glob("*"))[:10])

PROJECT_DIR: /content/drive/MyDrive/pill_det/mnt_data
존재? True
안의 파일 10개 예시: [PosixPath('/content/drive/MyDrive/pill_det/mnt_data/pred_ensemble_1761707940.csv'), PosixPath('/content/drive/MyDrive/pill_det/mnt_data/submission_ensemble_ocr.csv'), PosixPath('/content/drive/MyDrive/pill_det/mnt_data/all_results_csv.zip'), PosixPath('/content/drive/MyDrive/pill_det/mnt_data/submission_final.csv'), PosixPath('/content/drive/MyDrive/pill_det/mnt_data/results.csv'), PosixPath('/content/drive/MyDrive/pill_det/mnt_data/loss.zip')]


In [18]:
# =========================================================
# 🔁 데이터 교체·검증·통합 — 단일 셀
#   - DATA_CONFIG만 수정하면 새로운 파일로 df_master 재생성
#   - 제출CSV/YOLO results/FRCNN loss/K-Fold 묶음 ZIP 모두 선택적으로 처리
# =========================================================
import os, zipfile, json, time, re
from pathlib import Path
import numpy as np
import pandas as pd

# ---------------------------
# 0) 여기를 네 환경으로 교체
# ---------------------------
DATA_CONFIG = {
    # 베이스 폴더 (Colab: 구글드라이브 or 로컬)
    "BASE_DIR": "/content/drive/MyDrive/pill_det/mnt_data",

    # 파일 경로들 (없으면 None 허용)
    "PATH_SUB_FINAL": "submission_final.csv",          # OCR 재랭킹 최종 제출
    "PATH_SUB_ENS":   "submission_ensemble_ocr.csv",   # 앙상블+OCR 제출
    "PATH_FINAL_RES": "results.csv",                   # 최종 overfit run results.csv
    "PATH_KFOLD_ZIP": "all_results_csv.zip",          # K-Fold results.csv 묶음 ZIP
    "PATH_LOSS_ZIP":  "loss.zip",                      # FRCNN loss json 묶음 ZIP

    # 출력
    "OUT_DIR": "out",                                  # BASE_DIR/out
    "MASTER_NAME": "analysis_master.csv",              # 마스터 CSV 파일명
}

# ---------------------------
# 1) 경로 유틸
# ---------------------------
BASE_DIR   = DATA_CONFIG["BASE_DIR"]
OUT_DIR    = os.path.join(BASE_DIR, DATA_CONFIG["OUT_DIR"])
MASTER_CSV = os.path.join(OUT_DIR, DATA_CONFIG["MASTER_NAME"])
os.makedirs(OUT_DIR, exist_ok=True)

def _p(rel_or_abs):
    """BASE_DIR 기준 상대/절대 경로를 안전하게 병합"""
    if rel_or_abs is None:
        return None
    p = str(rel_or_abs)
    return p if p.startswith("/") or p.startswith("\\") else os.path.join(BASE_DIR, p)

# 실제 경로 확정
PATH_SUB_FINAL = _p(DATA_CONFIG["PATH_SUB_FINAL"])
PATH_SUB_ENS   = _p(DATA_CONFIG["PATH_SUB_ENS"])
PATH_FINAL_RES = _p(DATA_CONFIG["PATH_FINAL_RES"])
PATH_KFOLD_ZIP = _p(DATA_CONFIG["PATH_KFOLD_ZIP"])
PATH_LOSS_ZIP  = _p(DATA_CONFIG["PATH_LOSS_ZIP"])

print("🔧 BASE:", BASE_DIR)
for k, v in {
    "PATH_SUB_FINAL": PATH_SUB_FINAL,
    "PATH_SUB_ENS": PATH_SUB_ENS,
    "PATH_FINAL_RES": PATH_FINAL_RES,
    "PATH_KFOLD_ZIP": PATH_KFOLD_ZIP,
    "PATH_LOSS_ZIP": PATH_LOSS_ZIP,
    "OUT_DIR": OUT_DIR,
    "MASTER_CSV": MASTER_CSV,
}.items():
    print(f" - {k}: {v}")

# ---------------------------
# 2) 로더 & 파서
# ---------------------------
def safe_read_csv(path, **kwargs):
    if not path or not os.path.isfile(path):
        return None
    try:
        return pd.read_csv(path, **kwargs)
    except Exception:
        pass
    for sep in [",", ";", "\t", " "]:
        try:
            return pd.read_csv(path, sep=sep)
        except Exception:
            continue
    for enc in ["utf-8-sig","cp949","euc-kr","latin-1"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            continue
    return None

def extract_all_from_zip(zip_path, target_dir, exts=None):
    if not zip_path or not os.path.isfile(zip_path):
        return []
    os.makedirs(target_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(target_dir)
    paths = list(Path(target_dir).glob("**/*"))
    if exts:
        paths = [str(p) for p in paths if p.suffix.lower() in exts]
    else:
        paths = [str(p) for p in paths]
    return paths

def parse_submission_df(df_sub: pd.DataFrame, prefix: str):
    if df_sub is None:
        return None, None
    # 유연한 컬럼명 감지
    cols = {c.lower(): c for c in df_sub.columns}
    if ("image_id" not in cols.values()) or ("predictionstring" not in [c.lower() for c in df_sub.columns]):
        return None, None
    img_col = next(c for c in df_sub.columns if c.lower()=="image_id")
    ps_col  = next(c for c in df_sub.columns if c.lower()=="predictionstring")

    rows_stat, rows_flat = [], []
    for _, r in df_sub.iterrows():
        img_id = str(r[img_col])
        pred   = r[ps_col]
        num_boxes=0; avg_score=max_score=min_score=0.0
        scores=[]
        if isinstance(pred, str) and pred.strip():
            parts = pred.split()
            chunks = [parts[i:i+6] for i in range(0, len(parts), 6)]
            for ch in chunks:
                if len(ch) < 6:
                    continue
                cname = ch[0]
                try: s = float(ch[1])
                except: s = np.nan
                try: x1,y1,x2,y2 = map(float, ch[2:6])
                except: x1=y1=x2=y2=np.nan
                rows_flat.append({
                    "image_id": img_id, "cls_name": cname, "score": s,
                    "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                    "source_submission": prefix
                })
                scores.append(s)
            num_boxes = len(chunks)
            if scores:
                avg_score = float(np.nanmean(scores))
                max_score = float(np.nanmax(scores))
                min_score = float(np.nanmin(scores))
        rows_stat.append({
            "image_id": img_id,
            f"num_boxes_{prefix}": num_boxes,
            f"avg_score_{prefix}": avg_score,
            f"max_score_{prefix}": max_score,
            f"min_score_{prefix}": min_score,
        })
    return pd.DataFrame(rows_stat).drop_duplicates("image_id"), (pd.DataFrame(rows_flat) if rows_flat else None)

def pick_metric_column(cols):
    for c in cols:
        lc = c.lower()
        if "map" in lc and ("50-95" in lc or "50:95" in lc):
            return c
    for c in cols:
        if "map" in c.lower() and "50" in c.lower():
            return c
    for c in cols:
        if "map" in c.lower():
            return c
    return None

def guess_fold_and_model(run_name: str):
    fold, model = None, None
    m = re.match(r"^fold(\d+)_([A-Za-z0-9\-]+)", run_name or "")
    if m:
        try: fold = int(m.group(1))
        except: pass
        model_plus = m.group(2)
        for cand in ["yolov8n","yolov8s","yolov8m","yolov8l","yolov8x","yolo11n","yolo11s","yolo11m","yolo11l","yolo11x"]:
            if model_plus.startswith(cand):
                model = cand
                break
        if model is None: model = model_plus
    return fold, model

def parse_yolo_results(df_res: pd.DataFrame, run_name: str, phase: str, fold_guess=None, model_type_guess=None):
    if df_res is None or len(df_res)==0:
        return None
    cols = list(df_res.columns)
    loss_cols = [c for c in cols if "loss" in c.lower()]
    map_col = pick_metric_column(cols)
    f_auto, m_auto = guess_fold_and_model(run_name)
    fold   = f_auto if fold_guess is None else fold_guess
    model  = m_auto if model_type_guess is None else model_type_guess

    out=[]
    for _, row in df_res.iterrows():
        ep = pd.to_numeric(row.get("epoch", np.nan), errors="coerce")
        loss_sum, seen = 0.0, False
        for lc in loss_cols:
            try:
                v = float(row[lc])
                if not np.isnan(v):
                    loss_sum += v; seen = True
            except: pass
        loss_sum = loss_sum if seen else np.nan
        m5095 = np.nan
        if map_col and map_col in row:
            try: m5095 = float(row[map_col])
            except: m5095 = np.nan
        out.append({
            "row_type":"train_epoch","phase":phase,"run_name":run_name,
            "fold":fold,"model_type":model,"epoch":ep,
            "train_loss":loss_sum,"val_loss":np.nan,"map5095":m5095,
            "timestamp":time.time(),
            "image_id":np.nan,"num_boxes_final":np.nan,"avg_score_final":np.nan,"max_score_final":np.nan,"min_score_final":np.nan,
            "num_boxes_ens":np.nan,"avg_score_ens":np.nan,"max_score_ens":np.nan,"min_score_ens":np.nan,
            "score_gap":np.nan,"boxcount_gap":np.nan
        })
    return pd.DataFrame(out)

def parse_frcnn_jsonloss(df_json: pd.DataFrame, run_name: str, fold_guess=None):
    if df_json is None or len(df_json)==0:
        return None
    m = re.search(r"fold(\d+)", run_name or "")
    fold = fold_guess
    if fold is None and m:
        try: fold = int(m.group(1))
        except: pass
    out=[]
    for _, row in df_json.iterrows():
        out.append({
            "row_type":"train_epoch","phase":"frcnn","run_name":run_name,"fold":fold,
            "model_type":"fasterrcnn_resnet50_fpn",
            "epoch":row.get("epoch", np.nan),
            "train_loss":row.get("train_loss", np.nan),
            "val_loss":row.get("val_loss", np.nan),
            "map5095":np.nan,"timestamp":row.get("time", time.time()),
            "image_id":np.nan,"num_boxes_final":np.nan,"avg_score_final":np.nan,"max_score_final":np.nan,"min_score_final":np.nan,
            "num_boxes_ens":np.nan,"avg_score_ens":np.nan,"max_score_ens":np.nan,"min_score_ens":np.nan,
            "score_gap":np.nan,"boxcount_gap":np.nan
        })
    return pd.DataFrame(out)

# ---------------------------
# 3) 데이터 로드
# ---------------------------
df_sub_final = safe_read_csv(PATH_SUB_FINAL)
df_sub_ens   = safe_read_csv(PATH_SUB_ENS)
df_final_run = safe_read_csv(PATH_FINAL_RES)

print("\n📦 원본 형태")
print(" - submission_final:", None if df_sub_final is None else df_sub_final.shape)
print(" - submission_ens  :", None if df_sub_ens   is None else df_sub_ens.shape)
print(" - final results   :", None if df_final_run is None else df_final_run.shape)

EXTRACT_KFOLD_DIR = os.path.join(OUT_DIR, "kfold_extracted")
EXTRACT_LOSS_DIR  = os.path.join(OUT_DIR, "loss_extracted")

kfold_csv_paths = extract_all_from_zip(PATH_KFOLD_ZIP, EXTRACT_KFOLD_DIR, exts={".csv"})
loss_json_paths = extract_all_from_zip(PATH_LOSS_ZIP,  EXTRACT_LOSS_DIR,  exts={".json"})

df_kfold_all = []
for p in kfold_csv_paths:
    tmp = safe_read_csv(p)
    if tmp is not None and len(tmp):
        tmp["__src_file"] = Path(p).name
        df_kfold_all.append(tmp)
df_kfold_all = pd.concat(df_kfold_all, ignore_index=True) if df_kfold_all else None

df_loss_all = []
for p in loss_json_paths:
    try:
        tmp = pd.read_json(p)
        tmp["__src_file"] = Path(p).name
        df_loss_all.append(tmp)
    except: pass
df_loss_all = pd.concat(df_loss_all, ignore_index=True) if df_loss_all else None

print(" - K-Fold zip     :", None if df_kfold_all is None else df_kfold_all.shape)
print(" - FRCNN loss zip :", None if df_loss_all  is None else df_loss_all.shape)

# ---------------------------
# 4) 제출 CSV → 이미지별 통계
# ---------------------------
sub_final_stats, sub_final_flat = parse_submission_df(df_sub_final, prefix="final")
sub_ens_stats,   sub_ens_flat   = parse_submission_df(df_sub_ens,   prefix="ens")

if sub_final_stats is None and sub_ens_stats is None:
    df_sub_merged = pd.DataFrame()
else:
    if sub_final_stats is None:
        df_sub_merged = sub_ens_stats.copy()
    elif sub_ens_stats is None:
        df_sub_merged = sub_final_stats.copy()
    else:
        df_sub_merged = sub_final_stats.merge(sub_ens_stats, on="image_id", how="outer")
    # 파생치
    if "avg_score_final" in df_sub_merged.columns and "avg_score_ens" in df_sub_merged.columns:
        df_sub_merged["score_gap"] = df_sub_merged["avg_score_ens"] - df_sub_merged["avg_score_final"]
    if "num_boxes_final" in df_sub_merged.columns and "num_boxes_ens" in df_sub_merged.columns:
        df_sub_merged["boxcount_gap"] = df_sub_merged["num_boxes_ens"] - df_sub_merged["num_boxes_final"]

def wrap_submission_rows(df_in):
    if df_in is None or len(df_in)==0:
        return pd.DataFrame()
    out = df_in.copy()
    out["row_type"]="submission_image"
    out["phase"]="inference"
    out["run_name"]="compare_final_vs_ens"
    out["fold"]=np.nan
    out["model_type"]="inference_compare"
    out["epoch"]=np.nan
    out["train_loss"]=np.nan
    out["val_loss"]=np.nan
    out["map5095"]=np.nan
    out["timestamp"]=time.time()
    for col in [
        "num_boxes_final","avg_score_final","max_score_final","min_score_final",
        "num_boxes_ens","avg_score_ens","max_score_ens","min_score_ens",
        "score_gap","boxcount_gap"
    ]:
        if col not in out.columns: out[col]=np.nan
    return out.reset_index(drop=True)

df_sub_rows = wrap_submission_rows(df_sub_merged)
print("\n🧮 df_sub_rows:", df_sub_rows.shape)

# ---------------------------
# 5) 학습 로그 표준화
# ---------------------------
rows_training=[]

# K-Fold YOLO
if df_kfold_all is not None:
    for run_src, part in df_kfold_all.groupby("__src_file"):
        rn = Path(run_src).stem
        dfp = parse_yolo_results(part, run_name=rn, phase="kfold")
        if dfp is not None and len(dfp): rows_training.append(dfp)

# Final overfit YOLO
if df_final_run is not None and len(df_final_run):
    dfp = parse_yolo_results(df_final_run, run_name="final_overfit", phase="final_overfit")
    if dfp is not None and len(dfp): rows_training.append(dfp)

# FRCNN
if df_loss_all is not None and len(df_loss_all):
    for src, part in df_loss_all.groupby("__src_file"):
        rn = Path(src).stem
        dfp = parse_frcnn_jsonloss(part, run_name=rn)
        if dfp is not None and len(dfp): rows_training.append(dfp)

df_training_all = pd.concat(rows_training, ignore_index=True) if rows_training else pd.DataFrame()
print("🧪 df_training_all:", df_training_all.shape)

# ---------------------------
# 6) 마스터 결합 & 저장
# ---------------------------
df_master = pd.concat([df_training_all, df_sub_rows], ignore_index=True)

ordered = [
    "row_type","phase","run_name","fold","model_type","epoch",
    "train_loss","val_loss","map5095","timestamp","image_id",
    "num_boxes_final","avg_score_final","max_score_final","min_score_final",
    "num_boxes_ens","avg_score_ens","max_score_ens","min_score_ens",
    "score_gap","boxcount_gap"
]
for c in ordered:
    if c not in df_master.columns: df_master[c]=np.nan
df_master = df_master[ordered + [c for c in df_master.columns if c not in ordered]]

df_master.to_csv(MASTER_CSV, index=False, encoding="utf-8-sig")

print("\n✅ DONE")
print(" - MASTER_CSV:", MASTER_CSV, "shape:", df_master.shape)
print(" - 샘플 미리보기:")
display(df_master.head(10))

🔧 BASE: /content/drive/MyDrive/pill_det/mnt_data
 - PATH_SUB_FINAL: /content/drive/MyDrive/pill_det/mnt_data/submission_final.csv
 - PATH_SUB_ENS: /content/drive/MyDrive/pill_det/mnt_data/submission_ensemble_ocr.csv
 - PATH_FINAL_RES: /content/drive/MyDrive/pill_det/mnt_data/results.csv
 - PATH_KFOLD_ZIP: /content/drive/MyDrive/pill_det/mnt_data/all_results_csv.zip
 - PATH_LOSS_ZIP: /content/drive/MyDrive/pill_det/mnt_data/loss.zip
 - OUT_DIR: /content/drive/MyDrive/pill_det/mnt_data/out
 - MASTER_CSV: /content/drive/MyDrive/pill_det/mnt_data/out/analysis_master.csv

📦 원본 형태
 - submission_final: (3233, 8)
 - submission_ens  : (3341, 8)
 - final results   : (300, 15)
 - K-Fold zip     : (791, 16)
 - FRCNN loss zip : (7140, 7)

🧮 df_sub_rows: (0, 0)
🧪 df_training_all: (8231, 21)

✅ DONE
 - MASTER_CSV: /content/drive/MyDrive/pill_det/mnt_data/out/analysis_master.csv shape: (8231, 21)
 - 샘플 미리보기:


,row_type,phase,run_name,fold,model_type,epoch,train_loss,val_loss,map5095,timestamp,...,num_boxes_final,avg_score_final,max_score_final,min_score_final,num_boxes_ens,avg_score_ens,max_score_ens,min_score_ens,score_gap,boxcount_gap
0,train_epoch,kfold,results,None,None,1.0,20.46230,NaN,0.00935,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,train_epoch,kfold,results,None,None,2.0,12.50269,NaN,0.08987,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,train_epoch,kfold,results,None,None,3.0,8.07739,NaN,0.33295,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,train_epoch,kfold,results,None,None,4.0,6.01768,NaN,0.58544,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,train_epoch,kfold,results,None,None,5.0,5.11035,NaN,0.72451,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,train_epoch,kfold,results,None,None,6.0,4.64403,NaN,0.76662,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,train_epoch,kfold,results,None,None,7.0,4.25749,NaN,0.79502,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,train_epoch,kfold,results,None,None,8.0,3.95353,NaN,0.83328,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,train_epoch,kfold,results,None,None,9.0,3.81607,NaN,0.83547,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,train_epoch,kfold,results,None,None,10.0,3.65787,NaN,0.85454,1.762152e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# ============================
# ONE-CELL: 통합 로드 → 정규화 → 시각화(저장X)
# ============================
import os, zipfile, json, time, re, math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --------------------------------
# 0) 경로 설정 (필요시 여기를 바꿔)
# --------------------------------
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

DRIVE_BASE = "/content/drive/MyDrive/pill_det/mnt_data"
PATH_SUB_FINAL = f"{DRIVE_BASE}/submission_final.csv"
PATH_SUB_ENS   = f"{DRIVE_BASE}/submission_ensemble_ocr.csv"
PATH_FINAL_RES = f"{DRIVE_BASE}/results.csv"
PATH_KFOLD_ZIP = f"{DRIVE_BASE}/all_results_csv.zip"
PATH_LOSS_ZIP  = f"{DRIVE_BASE}/loss.zip"

print("BASE:", DRIVE_BASE)
print(" - PATH_SUB_FINAL:", PATH_SUB_FINAL)
print(" - PATH_SUB_ENS:", PATH_SUB_ENS)
print(" - PATH_FINAL_RES:", PATH_FINAL_RES)
print(" - PATH_KFOLD_ZIP:", PATH_KFOLD_ZIP)
print(" - PATH_LOSS_ZIP:", PATH_LOSS_ZIP)

# --------------------------------
# 1) 유틸
# --------------------------------
def safe_read_csv(path, **kwargs):
    if not os.path.isfile(path):
        return None
    try:
        return pd.read_csv(path, **kwargs)
    except Exception:
        for sep in [",",";","\t"," "]:
            try:
                return pd.read_csv(path, sep=sep)
            except Exception:
                continue
        for enc in ["utf-8-sig","cp949","euc-kr","latin-1"]:
            try:
                return pd.read_csv(path, encoding=enc)
            except Exception:
                continue
    return None

def extract_all_from_zip(zip_path, exts=None):
    if not os.path.isfile(zip_path):
        return []
    out_dir = os.path.join(os.path.dirname(zip_path), "_extracted_tmp")
    os.makedirs(out_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(out_dir)
    if exts is None:
        return [str(p) for p in Path(out_dir).glob("**/*")]
    out = []
    for ext in exts:
        out.extend(Path(out_dir).glob(f"**/*{ext}"))
    return [str(p) for p in out]

def pick_metric_column(cols):
    # YOLO results.csv mAP 우선순위
    for c in cols:
        if "mAP" in c and ("50-95" in c or "50:95" in c):
            return c
    for c in cols:
        if "mAP" in c and "50" in c:
            return c
    for c in cols:
        if "mAP" in c:
            return c
    return None

def parse_yolo_results(df_res: pd.DataFrame, run_name: str, phase: str):
    """YOLO results.csv → epoch별 표준화 (train_loss=box+cls+dfl 합)"""
    if df_res is None or len(df_res)==0:
        return None
    cols = list(df_res.columns)
    loss_cols = [c for c in cols if "loss" in c.lower()]
    map_col = pick_metric_column(cols)

    # run_name에서 fold, model 유추(있으면)
    fold, model = None, None
    m = re.match(r"^fold(\d+)_([A-Za-z0-9\-]+)", run_name)
    if m:
        try:
            fold = int(m.group(1))
        except:
            fold = None
        model = m.group(2)

    rows = []
    for _, row in df_res.iterrows():
        ep = pd.to_numeric(row.get("epoch", np.nan), errors="coerce")
        # loss 합
        loss_sum, saw = 0.0, False
        for lc in loss_cols:
            v = pd.to_numeric(row.get(lc, np.nan), errors="coerce")
            if pd.notna(v):
                loss_sum += float(v)
                saw = True
        if not saw:
            loss_sum = np.nan
        # mAP
        m5095 = np.nan
        if map_col and map_col in row:
            m5095 = pd.to_numeric(row[map_col], errors="coerce")

        rows.append({
            "row_type": "train_epoch",
            "phase": phase,
            "run_name": run_name,
            "fold": fold,
            "model_type": model,
            "epoch": float(ep) if pd.notna(ep) else np.nan,
            "train_loss": loss_sum,
            "val_loss": np.nan,
            "map5095": float(m5095) if pd.notna(m5095) else np.nan,
            "timestamp": time.time(),
            # submission 비교용 칼럼들은 빈칸
            "image_id": np.nan,
            "num_boxes_final": np.nan, "avg_score_final": np.nan, "max_score_final": np.nan, "min_score_final": np.nan,
            "num_boxes_ens": np.nan,   "avg_score_ens": np.nan,   "max_score_ens": np.nan,   "min_score_ens": np.nan,
            "score_gap": np.nan, "boxcount_gap": np.nan
        })
    return pd.DataFrame(rows)

def parse_submission_df(df_sub: pd.DataFrame, prefix: str):
    """제출 CSV(image_id, PredictionString) → 이미지별 통계/평탄화"""
    if df_sub is None or "image_id" not in df_sub.columns or "PredictionString" not in df_sub.columns:
        return None, None
    rows_stat, rows_flat = [], []
    for _, r in df_sub.iterrows():
        img_id = str(r["image_id"])
        pred = r["PredictionString"]
        nbox = 0
        avg_sc = max_sc = min_sc = 0.0
        scores = []
        if isinstance(pred, str) and pred.strip():
            parts = pred.strip().split()
            chunks = [parts[i:i+6] for i in range(0, len(parts), 6)]
            for ch in chunks:
                if len(ch) < 6:
                    continue
                cname = ch[0]
                s = pd.to_numeric(ch[1], errors="coerce")
                try:
                    x1,y1,x2,y2 = map(float, ch[2:6])
                except:
                    x1=y1=x2=y2=np.nan
                rows_flat.append({
                    "image_id": img_id,
                    "cls_name": cname,
                    "score": float(s) if pd.notna(s) else np.nan,
                    "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                    "source_submission": prefix
                })
                if pd.notna(s): scores.append(float(s))
            nbox = len(chunks)
            if scores:
                avg_sc = float(np.nanmean(scores))
                max_sc = float(np.nanmax(scores))
                min_sc = float(np.nanmin(scores))
        rows_stat.append({
            "image_id": img_id,
            f"num_boxes_{prefix}": nbox,
            f"avg_score_{prefix}": avg_sc,
            f"max_score_{prefix}": max_sc,
            f"min_score_{prefix}": min_sc,
        })
    return pd.DataFrame(rows_stat).drop_duplicates(subset=["image_id"]), (pd.DataFrame(rows_flat) if rows_flat else None)

def load_json_rows_from_zip(zip_path: str):
    """loss.zip 안 여러 JSON → 단일 DF (epoch/mAP/box,cls,dfl, train_loss 합산)"""
    if not os.path.isfile(zip_path):
        return None
    rows = []
    with zipfile.ZipFile(zip_path, 'r') as z:
        for name in z.namelist():
            if not name.lower().endswith(".json"):
                continue
            with z.open(name) as f:
                try:
                    data = json.load(f)
                except Exception:
                    continue
            run_name = Path(name).stem
            m = re.search(r"fold(\d+)", run_name)
            fold = int(m.group(1)) if m else None

            if isinstance(data, list):
                recs = data
            elif isinstance(data, dict) and "history" in data and isinstance(data["history"], list):
                recs = data["history"]
            else:
                recs = [data]

            for r in recs:
                epoch = pd.to_numeric(r.get("epoch", np.nan), errors="coerce")
                box_loss = pd.to_numeric(r.get("box_loss", np.nan), errors="coerce")
                cls_loss = pd.to_numeric(r.get("cls_loss", np.nan), errors="coerce")
                dfl_loss = pd.to_numeric(r.get("dfl_loss", np.nan), errors="coerce")
                # mAP 표기 다양성 대응
                m5095 = r.get("mAP_0.5_0.95", r.get("mAP_0.5-0.95", r.get("metrics/mAP50-95(B)", np.nan)))
                m5095 = pd.to_numeric(m5095, errors="coerce")
                m50 = pd.to_numeric(r.get("mAP_0.5", np.nan), errors="coerce")
                loss_sum = np.nan
                parts = [box_loss, cls_loss, dfl_loss]
                if np.isfinite(parts).all():
                    loss_sum = float(np.nansum(parts))
                rows.append({
                    "row_type": "train_epoch",
                    "phase": "frcnn_or_yolojson",
                    "run_name": run_name,
                    "fold": fold,
                    "model_type": None,
                    "epoch": float(epoch) if pd.notna(epoch) else np.nan,
                    "train_loss": loss_sum,
                    "val_loss": np.nan,
                    "map5095": float(m5095) if pd.notna(m5095) else np.nan,
                    "timestamp": time.time(),
                    "image_id": np.nan,
                    "num_boxes_final": np.nan, "avg_score_final": np.nan, "max_score_final": np.nan, "min_score_final": np.nan,
                    "num_boxes_ens": np.nan,   "avg_score_ens": np.nan,   "max_score_ens": np.nan,   "min_score_ens": np.nan,
                    "score_gap": np.nan, "boxcount_gap": np.nan,
                    "mAP50": float(m50) if pd.notna(m50) else np.nan,
                })
    return pd.DataFrame(rows) if rows else None

# --------------------------------
# 2) 원천 로드
# --------------------------------
df_sub_final = safe_read_csv(PATH_SUB_FINAL)
df_sub_ens   = safe_read_csv(PATH_SUB_ENS)
df_final_run = safe_read_csv(PATH_FINAL_RES)

# ZIP 안 K-Fold CSV들
kfold_csv_paths = extract_all_from_zip(PATH_KFOLD_ZIP, exts=[".csv"])
df_kfold_all = []
for p in kfold_csv_paths:
    tmp = safe_read_csv(p)
    if tmp is not None and len(tmp):
        tmp["__src_file"] = Path(p).name
        df_kfold_all.append(tmp)
df_kfold_all = pd.concat(df_kfold_all, ignore_index=True) if df_kfold_all else None

# loss.zip (여러 json)
df_loss_zip = load_json_rows_from_zip(PATH_LOSS_ZIP)

print("원본 크기 체크")
print(" - submission_final:", None if df_sub_final is None else df_sub_final.shape)
print(" - submission_ens  :", None if df_sub_ens   is None else df_sub_ens.shape)
print(" - final results   :", None if df_final_run is None else df_final_run.shape)
print(" - K-Fold zip      :", None if df_kfold_all is None else df_kfold_all.shape)
print(" - loss.zip (json) :", None if df_loss_zip  is None else df_loss_zip.shape)

# --------------------------------
# 3) 제출 CSV → 이미지별 통계 병합
# --------------------------------
sub_final_stats, _ = parse_submission_df(df_sub_final, prefix="final")
sub_ens_stats,   _ = parse_submission_df(df_sub_ens,   prefix="ens")

if sub_final_stats is None and sub_ens_stats is None:
    df_sub_merged = pd.DataFrame()
else:
    if sub_final_stats is None:
        df_sub_merged = sub_ens_stats.copy()
    elif sub_ens_stats is None:
        df_sub_merged = sub_final_stats.copy()
    else:
        df_sub_merged = sub_final_stats.merge(sub_ens_stats, on="image_id", how="outer")

    if "avg_score_final" in df_sub_merged.columns and "avg_score_ens" in df_sub_merged.columns:
        df_sub_merged["score_gap"] = df_sub_merged["avg_score_ens"] - df_sub_merged["avg_score_final"]
    if "num_boxes_final" in df_sub_merged.columns and "num_boxes_ens" in df_sub_merged.columns:
        df_sub_merged["boxcount_gap"] = df_sub_merged["num_boxes_ens"] - df_sub_merged["num_boxes_final"]

def wrap_submission_rows(df_in):
    if df_in is None or len(df_in)==0:
        return pd.DataFrame()
    out = df_in.copy()
    out["row_type"]   = "submission_image"
    out["phase"]      = "inference"
    out["run_name"]   = "compare_final_vs_ens"
    out["fold"]       = np.nan
    out["model_type"] = "inference_compare"
    out["epoch"]      = np.nan
    out["train_loss"] = np.nan
    out["val_loss"]   = np.nan
    out["map5095"]    = np.nan
    out["timestamp"]  = time.time()
    for col in [
        "num_boxes_final","avg_score_final","max_score_final","min_score_final",
        "num_boxes_ens","avg_score_ens","max_score_ens","min_score_ens",
        "score_gap","boxcount_gap"
    ]:
        if col not in out.columns:
            out[col] = np.nan
    return out.reset_index(drop=True)

df_sub_rows = wrap_submission_rows(df_sub_merged)

# --------------------------------
# 4) 학습 로그 표준화 & 결합 (K-Fold CSV + Final CSV + loss.zip JSON)
# --------------------------------
rows_training = []

# K-Fold: 각 파일(__src_file)별로 run_name 추정해서 parse
if df_kfold_all is not None:
    for src, part in df_kfold_all.groupby("__src_file"):
        rn = Path(src).stem
        dfp = parse_yolo_results(part, run_name=rn, phase="kfold")
        if dfp is not None and len(dfp):
            rows_training.append(dfp)

# Final results.csv
if df_final_run is not None and len(df_final_run):
    rn_final = "final_overfit"  # 이름은 자유
    dfp = parse_yolo_results(df_final_run, run_name=rn_final, phase="final_overfit")
    if dfp is not None and len(dfp):
        rows_training.append(dfp)

# loss.zip JSON 통합(여러 run 포함 가능)
if df_loss_zip is not None and len(df_loss_zip):
    rows_training.append(df_loss_zip)

df_training_all = pd.concat(rows_training, ignore_index=True) if rows_training else pd.DataFrame()

# Master 합치기
df_master = pd.concat([df_training_all, df_sub_rows], ignore_index=True)
print("\n합쳐진 df_master:", df_master.shape)

# --------------------------------
# 5) 그래프(저장X) — 한 줄 3개씩
#    (A) Epoch vs mAP50-95 (line, run별)
#    (B) Epoch vs train_loss (line, run별)  ※ loss는 YOLO 합(box+cls+dfl)
#    (C) 제출 비교: score_gap, boxcount_gap 히스토그램
#    (D) 제출 비교: avg_score_final vs avg_score_ens, num_boxes_final vs num_boxes_ens 산점도
# --------------------------------

def three_per_row(runs):
    """runs 리스트를 3개씩 끊는 제너레이터"""
    for i in range(0, len(runs), 3):
        yield runs[i:i+3]

# (A) Epoch vs mAP50-95
df_train_map = df_master[(df_master["row_type"]=="train_epoch") & df_master["map5095"].notna()]
if len(df_train_map):
    run_list = list(df_train_map["run_name"].dropna().unique())
    for group in three_per_row(run_list):
        plt.figure(figsize=(18,4))
        for i, rn in enumerate(group, start=1):
            part = df_train_map[df_train_map["run_name"]==rn].dropna(subset=["epoch","map5095"])
            plt.subplot(1,3,i)
            plt.plot(part["epoch"], part["map5095"], marker="o")
            plt.title(f"[{rn}] Epoch vs mAP50-95")
            plt.xlabel("Epoch"); plt.ylabel("mAP50-95"); plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# (B) Epoch vs train_loss (YOLO 합)
df_train_loss = df_master[(df_master["row_type"]=="train_epoch") & df_master["train_loss"].notna()]
if len(df_train_loss):
    run_list = list(df_train_loss["run_name"].dropna().unique())
    for group in three_per_row(run_list):
        plt.figure(figsize=(18,4))
        for i, rn in enumerate(group, start=1):
            part = df_train_loss[df_train_loss["run_name"]==rn].dropna(subset=["epoch","train_loss"])
            plt.subplot(1,3,i)
            plt.plot(part["epoch"], part["train_loss"], marker="o")
            plt.title(f"[{rn}] Epoch vs train_loss")
            plt.xlabel("Epoch"); plt.ylabel("train_loss"); plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# (C) 제출 비교 히스토그램 (OCR+ENS - FINAL)
df_inf = df_master[df_master["row_type"]=="submission_image"]
if len(df_inf):
    # score gap
    if "score_gap" in df_inf.columns and df_inf["score_gap"].notna().any():
        plt.figure(figsize=(6,4))
        plt.hist(df_inf["score_gap"].dropna(), bins=40)
        plt.title("Avg Score Gap (Ensemble+OCR - Final)")
        plt.xlabel("avg_score_ens - avg_score_final"); plt.ylabel("count")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    # boxcount gap
    if "boxcount_gap" in df_inf.columns and df_inf["boxcount_gap"].notna().any():
        plt.figure(figsize=(6,4))
        plt.hist(df_inf["boxcount_gap"].dropna(), bins=40)
        plt.title("Box Count Gap (Ensemble - Final)")
        plt.xlabel("num_boxes_ens - num_boxes_final"); plt.ylabel("count")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# (D) 제출 비교 산점도
if len(df_inf):
    # avg score scatter
    if set(["avg_score_final","avg_score_ens"]).issubset(df_inf.columns):
        s = df_inf[["avg_score_final","avg_score_ens"]].dropna()
        if len(s):
            plt.figure(figsize=(5,5))
            plt.scatter(s["avg_score_final"], s["avg_score_ens"], s=8)
            lim = [min(s.min())*0.95, max(s.max())*1.05]
            plt.plot(lim, lim)  # y=x 기준선
            plt.xlim(lim); plt.ylim(lim)
            plt.xlabel("avg_score_final"); plt.ylabel("avg_score_ens")
            plt.title("Avg Score: Final vs Ensemble+OCR")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
    # box count scatter
    if set(["num_boxes_final","num_boxes_ens"]).issubset(df_inf.columns):
        s = df_inf[["num_boxes_final","num_boxes_ens"]].dropna()
        if len(s):
            plt.figure(figsize=(5,5))
            plt.scatter(s["num_boxes_final"], s["num_boxes_ens"], s=8)
            lim = [min(s.min())*0.95, max(s.max())*1.05]
            plt.plot(lim, lim)
            plt.xlim(lim); plt.ylim(lim)
            plt.xlabel("num_boxes_final"); plt.ylabel("num_boxes_ens")
            plt.title("Box Count: Final vs Ensemble")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()

print("\n🎯 그래프 표시 완료")

Output hidden; open in https://colab.research.google.com to view.